# Experiment 020: Asymmetric Packing

The strategy identifies that all SA-based approaches converge to the same local optimum (~70.63).
The key unexplored direction is **asymmetric solutions** - generating initial configurations with:
- Random tree orientations (not just 0°, 90°, 180°, 270°)
- Non-grid placements (spiral, radial, random)
- Different tree densities in different regions

This experiment will:
1. Generate asymmetric initial configurations
2. Apply SA optimization to find new basins
3. Compare with baseline

In [1]:
import numpy as np
import pandas as pd
from shapely.geometry import Polygon
from shapely.affinity import rotate, translate
import random
from tqdm import tqdm
import json

# Tree template
TREE_TEMPLATE = [
    (0.0, 0.8), (0.125, 0.5), (0.0625, 0.5), (0.2, 0.25), (0.1, 0.25),
    (0.35, 0.0), (0.075, 0.0), (0.075, -0.2), (-0.075, -0.2), (-0.075, 0.0),
    (-0.35, 0.0), (-0.1, 0.25), (-0.2, 0.25), (-0.0625, 0.5), (-0.125, 0.5)
]

def create_tree_polygon(x, y, angle):
    """Create a tree polygon at position (x, y) with given rotation angle."""
    tree = Polygon(TREE_TEMPLATE)
    tree = rotate(tree, angle, origin=(0, 0), use_radians=False)
    tree = translate(tree, x, y)
    return tree

def check_overlap(tree1, tree2):
    """Check if two trees overlap (touching is allowed)."""
    return tree1.overlaps(tree2) or tree1.contains(tree2) or tree2.contains(tree1)

def get_bounding_box_side(trees):
    """Get the side length of the bounding box containing all trees."""
    all_x = []
    all_y = []
    for tree in trees:
        minx, miny, maxx, maxy = tree.bounds
        all_x.extend([minx, maxx])
        all_y.extend([miny, maxy])
    return max(max(all_x) - min(all_x), max(all_y) - min(all_y))

def calculate_score_contribution(side, n):
    """Calculate score contribution for a single N: s^2/n"""
    return (side ** 2) / n

print("Functions defined")

Functions defined


In [2]:
# Load baseline for comparison
def parse_s_value(val):
    if isinstance(val, str):
        if val.startswith('s'):
            return float(val[1:])
        return float(val)
    return float(val)

baseline_df = pd.read_csv('/home/code/exploration/datasets/ensemble_best.csv')
baseline_df['x'] = baseline_df['x'].apply(parse_s_value)
baseline_df['y'] = baseline_df['y'].apply(parse_s_value)
baseline_df['deg'] = baseline_df['deg'].apply(parse_s_value)
baseline_df['n'] = baseline_df['id'].apply(lambda x: int(x.split('_')[0]))

print(f"Baseline loaded: {len(baseline_df)} rows")

# Calculate baseline scores per N
baseline_scores = {}
for n in range(1, 201):
    group = baseline_df[baseline_df['n'] == n]
    trees = [create_tree_polygon(row['x'], row['y'], row['deg']) for _, row in group.iterrows()]
    side = get_bounding_box_side(trees)
    baseline_scores[n] = calculate_score_contribution(side, n)

baseline_total = sum(baseline_scores.values())
print(f"Baseline total score: {baseline_total:.6f}")

Baseline loaded: 20100 rows


Baseline total score: 70.630478


In [3]:
def generate_spiral_placement(n, scale=1.0):
    """Generate spiral placement for n trees."""
    positions = []
    angles = []
    
    # Golden angle for spiral
    golden_angle = 137.5077640500378
    
    for i in range(n):
        # Spiral radius grows with sqrt(i)
        r = scale * np.sqrt(i + 1) * 0.5
        # Angle based on golden ratio
        theta = i * golden_angle * np.pi / 180
        
        x = r * np.cos(theta)
        y = r * np.sin(theta)
        
        # Random rotation angle (continuous, not discrete)
        angle = random.uniform(0, 360)
        
        positions.append((x, y))
        angles.append(angle)
    
    return positions, angles

def generate_radial_placement(n, scale=1.0):
    """Generate radial placement with trees in concentric rings."""
    positions = []
    angles = []
    
    placed = 0
    ring = 0
    
    while placed < n:
        if ring == 0:
            # Center tree
            positions.append((0, 0))
            angles.append(random.uniform(0, 360))
            placed += 1
        else:
            # Trees in ring
            trees_in_ring = min(6 * ring, n - placed)
            for i in range(trees_in_ring):
                theta = 2 * np.pi * i / (6 * ring) + random.uniform(-0.1, 0.1)
                r = ring * scale * 0.8
                x = r * np.cos(theta)
                y = r * np.sin(theta)
                positions.append((x, y))
                angles.append(random.uniform(0, 360))
                placed += 1
                if placed >= n:
                    break
        ring += 1
    
    return positions, angles

def generate_random_placement(n, scale=1.0):
    """Generate random placement within a square."""
    positions = []
    angles = []
    
    # Estimate required area
    area_per_tree = 0.5  # Approximate tree footprint
    side = np.sqrt(n * area_per_tree) * scale
    
    for i in range(n):
        x = random.uniform(-side/2, side/2)
        y = random.uniform(-side/2, side/2)
        angle = random.uniform(0, 360)
        positions.append((x, y))
        angles.append(angle)
    
    return positions, angles

print("Placement generators defined")

Placement generators defined


In [4]:
def resolve_overlaps(positions, angles, max_iterations=1000):
    """Resolve overlaps by moving trees apart."""
    n = len(positions)
    positions = list(positions)
    angles = list(angles)
    
    for iteration in range(max_iterations):
        trees = [create_tree_polygon(x, y, a) for (x, y), a in zip(positions, angles)]
        
        # Find overlaps
        overlaps_found = False
        for i in range(n):
            for j in range(i+1, n):
                if check_overlap(trees[i], trees[j]):
                    overlaps_found = True
                    # Move trees apart
                    cx_i = trees[i].centroid.x
                    cy_i = trees[i].centroid.y
                    cx_j = trees[j].centroid.x
                    cy_j = trees[j].centroid.y
                    
                    dx = cx_j - cx_i
                    dy = cy_j - cy_i
                    dist = np.sqrt(dx*dx + dy*dy)
                    
                    if dist < 0.001:
                        dx, dy = random.uniform(-1, 1), random.uniform(-1, 1)
                        dist = np.sqrt(dx*dx + dy*dy)
                    
                    # Move both trees apart
                    move = 0.1 / dist
                    positions[i] = (positions[i][0] - dx * move, positions[i][1] - dy * move)
                    positions[j] = (positions[j][0] + dx * move, positions[j][1] + dy * move)
        
        if not overlaps_found:
            break
    
    return positions, angles

def optimize_sa(positions, angles, n, iterations=5000, t_start=1.0, t_end=0.001):
    """Simple SA optimization for a configuration."""
    positions = list(positions)
    angles = list(angles)
    
    # Create initial trees
    trees = [create_tree_polygon(x, y, a) for (x, y), a in zip(positions, angles)]
    
    # Check for overlaps
    def has_overlaps(trees):
        for i in range(len(trees)):
            for j in range(i+1, len(trees)):
                if check_overlap(trees[i], trees[j]):
                    return True
        return False
    
    if has_overlaps(trees):
        return None, None, float('inf')
    
    current_side = get_bounding_box_side(trees)
    best_side = current_side
    best_positions = positions.copy()
    best_angles = angles.copy()
    
    cooling_rate = (t_end / t_start) ** (1.0 / iterations)
    T = t_start
    
    for it in range(iterations):
        # Pick random tree
        idx = random.randint(0, n-1)
        
        # Random move
        move_type = random.choice(['translate', 'rotate', 'both'])
        
        old_pos = positions[idx]
        old_angle = angles[idx]
        
        if move_type in ['translate', 'both']:
            dx = random.gauss(0, 0.1 * T)
            dy = random.gauss(0, 0.1 * T)
            positions[idx] = (old_pos[0] + dx, old_pos[1] + dy)
        
        if move_type in ['rotate', 'both']:
            da = random.gauss(0, 10 * T)
            angles[idx] = (old_angle + da) % 360
        
        # Create new tree
        new_tree = create_tree_polygon(positions[idx][0], positions[idx][1], angles[idx])
        
        # Check collision
        collision = False
        for i in range(n):
            if i != idx:
                if check_overlap(new_tree, trees[i]):
                    collision = True
                    break
        
        if collision:
            positions[idx] = old_pos
            angles[idx] = old_angle
            T *= cooling_rate
            continue
        
        # Update tree
        trees[idx] = new_tree
        new_side = get_bounding_box_side(trees)
        
        # Accept or reject
        delta = new_side - current_side
        if delta < 0 or random.random() < np.exp(-delta / T):
            current_side = new_side
            if new_side < best_side:
                best_side = new_side
                best_positions = positions.copy()
                best_angles = angles.copy()
        else:
            positions[idx] = old_pos
            angles[idx] = old_angle
            trees[idx] = create_tree_polygon(old_pos[0], old_pos[1], old_angle)
        
        T *= cooling_rate
    
    return best_positions, best_angles, best_side

print("Optimization functions defined")

Optimization functions defined


In [5]:
# Test asymmetric approach on small N values first
print("Testing asymmetric approach on N=10, 20, 30...")

test_results = {}

for n in [10, 20, 30]:
    print(f"\nN={n}:")
    print(f"  Baseline score contribution: {baseline_scores[n]:.6f}")
    
    best_score = float('inf')
    best_method = None
    
    # Try different placement methods
    for method_name, generator in [('spiral', generate_spiral_placement), 
                                    ('radial', generate_radial_placement),
                                    ('random', generate_random_placement)]:
        
        # Multiple restarts
        for restart in range(5):
            random.seed(42 + restart + n * 100)
            np.random.seed(42 + restart + n * 100)
            
            # Generate initial placement
            positions, angles = generator(n, scale=1.0)
            
            # Resolve overlaps
            positions, angles = resolve_overlaps(positions, angles)
            
            # Optimize
            opt_pos, opt_angles, opt_side = optimize_sa(positions, angles, n, iterations=2000)
            
            if opt_side < float('inf'):
                score = calculate_score_contribution(opt_side, n)
                if score < best_score:
                    best_score = score
                    best_method = f"{method_name}_r{restart}"
    
    test_results[n] = {
        'baseline': baseline_scores[n],
        'asymmetric': best_score,
        'method': best_method,
        'improvement': baseline_scores[n] - best_score
    }
    
    print(f"  Best asymmetric score: {best_score:.6f} ({best_method})")
    print(f"  Improvement: {test_results[n]['improvement']:.6f}")

Testing asymmetric approach on N=10, 20, 30...

N=10:
  Baseline score contribution: 0.376630


  Best asymmetric score: 0.988287 (radial_r4)
  Improvement: -0.611657

N=20:
  Baseline score contribution: 0.376057


  Best asymmetric score: 1.019098 (random_r3)
  Improvement: -0.643041

N=30:
  Baseline score contribution: 0.360883


  Best asymmetric score: 0.957226 (random_r0)
  Improvement: -0.596343


In [6]:
# Analyze results
print("\n" + "="*50)
print("ASYMMETRIC APPROACH RESULTS")
print("="*50)

for n, result in test_results.items():
    status = "BETTER" if result['improvement'] > 0 else "WORSE"
    print(f"N={n}: Baseline={result['baseline']:.6f}, Asymmetric={result['asymmetric']:.6f}, {status} by {result['improvement']:.6f}")

# Check if any improvement
any_improvement = any(r['improvement'] > 0 for r in test_results.values())
print(f"\nAny improvement found: {any_improvement}")


ASYMMETRIC APPROACH RESULTS
N=10: Baseline=0.376630, Asymmetric=0.988287, WORSE by -0.611657
N=20: Baseline=0.376057, Asymmetric=1.019098, WORSE by -0.643041
N=30: Baseline=0.360883, Asymmetric=0.957226, WORSE by -0.596343

Any improvement found: False


In [7]:
# If asymmetric approach shows promise, scale up
# Otherwise, try a different strategy

if any_improvement:
    print("Asymmetric approach shows promise! Scaling up...")
    # Would continue with full optimization
else:
    print("Asymmetric approach did NOT improve on baseline.")
    print("The baseline configurations are already highly optimized.")
    print("\nThis confirms that the baseline is at a very strong local optimum.")
    print("The target (68.92) may require techniques beyond simple placement strategies.")

Asymmetric approach did NOT improve on baseline.
The baseline configurations are already highly optimized.

This confirms that the baseline is at a very strong local optimum.
The target (68.92) may require techniques beyond simple placement strategies.


In [8]:
# Save metrics regardless of outcome
import json

# Calculate what the ensemble score would be
ensemble_score = baseline_total  # Start with baseline
improvements_found = []

for n, result in test_results.items():
    if result['improvement'] > 0:
        ensemble_score -= result['improvement']
        improvements_found.append((n, result['improvement']))

metrics = {
    'cv_score': ensemble_score,
    'baseline_score': baseline_total,
    'test_results': test_results,
    'improvements_found': improvements_found,
    'any_improvement': any_improvement
}

with open('/home/code/experiments/020_asymmetric_packing/metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

print(f"Metrics saved. Ensemble score: {ensemble_score:.6f}")

Metrics saved. Ensemble score: 70.630478


In [9]:
# Save submission (use baseline since asymmetric didn't improve)
baseline_df_out = baseline_df[['id', 'x', 'y', 'deg']].copy()

def format_s_value(val):
    return f"s{val}"

baseline_df_out['x'] = baseline_df_out['x'].apply(format_s_value)
baseline_df_out['y'] = baseline_df_out['y'].apply(format_s_value)
baseline_df_out['deg'] = baseline_df_out['deg'].apply(format_s_value)

baseline_df_out.to_csv('/home/submission/submission.csv', index=False)
print("Saved baseline to submission.csv (asymmetric approach did not improve)")

print(f"\nFinal score: {ensemble_score:.6f}")
print(f"Target: 68.919154")
print(f"Gap: {ensemble_score - 68.919154:.6f} ({(ensemble_score - 68.919154) / 68.919154 * 100:.2f}%)")

Saved baseline to submission.csv (asymmetric approach did not improve)

Final score: 70.630478
Target: 68.919154
Gap: 1.711324 (2.48%)


In [10]:
# Let's try a more sophisticated approach: use the baseline positions but with random angle perturbations
print("\nTrying angle perturbation on baseline configurations...")

angle_results = {}

for n in [10, 20, 30, 50, 100]:
    group = baseline_df[baseline_df['n'] == n]
    baseline_positions = [(row['x'], row['y']) for _, row in group.iterrows()]
    baseline_angles = [row['deg'] for _, row in group.iterrows()]
    
    # Try perturbing angles
    best_score = baseline_scores[n]
    best_angles = baseline_angles.copy()
    
    for restart in range(10):
        random.seed(42 + restart + n * 100)
        
        # Perturb angles
        perturbed_angles = [(a + random.gauss(0, 30)) % 360 for a in baseline_angles]
        
        # Create trees
        trees = [create_tree_polygon(x, y, a) for (x, y), a in zip(baseline_positions, perturbed_angles)]
        
        # Check overlaps
        has_overlap = False
        for i in range(len(trees)):
            for j in range(i+1, len(trees)):
                if check_overlap(trees[i], trees[j]):
                    has_overlap = True
                    break
            if has_overlap:
                break
        
        if not has_overlap:
            side = get_bounding_box_side(trees)
            score = calculate_score_contribution(side, n)
            if score < best_score:
                best_score = score
                best_angles = perturbed_angles
    
    angle_results[n] = {
        'baseline': baseline_scores[n],
        'perturbed': best_score,
        'improvement': baseline_scores[n] - best_score
    }
    
    print(f"N={n}: Baseline={baseline_scores[n]:.6f}, Perturbed={best_score:.6f}, Improvement={angle_results[n]['improvement']:.6f}")

print("\nAngle perturbation results:")
any_angle_improvement = any(r['improvement'] > 0 for r in angle_results.values())
print(f"Any improvement: {any_angle_improvement}")


Trying angle perturbation on baseline configurations...
N=10: Baseline=0.376630, Perturbed=0.376630, Improvement=0.000000
N=20: Baseline=0.376057, Perturbed=0.376057, Improvement=0.000000
N=30: Baseline=0.360883, Perturbed=0.360883, Improvement=0.000000
N=50: Baseline=0.360753, Perturbed=0.360753, Improvement=0.000000


N=100: Baseline=0.343427, Perturbed=0.343427, Improvement=0.000000

Angle perturbation results:
Any improvement: False


In [11]:
# Let's try a different approach: analyze what makes the baseline so good
# and see if we can find any patterns

print("Analyzing baseline configuration patterns...")

# Look at angle distribution
all_angles = baseline_df['deg'].values
print(f"\nAngle statistics:")
print(f"  Min: {all_angles.min():.2f}")
print(f"  Max: {all_angles.max():.2f}")
print(f"  Mean: {all_angles.mean():.2f}")
print(f"  Std: {all_angles.std():.2f}")

# Check if angles are discrete or continuous
unique_angles = np.unique(np.round(all_angles, 1))
print(f"  Unique angles (rounded to 0.1): {len(unique_angles)}")

# Look at position distribution
all_x = baseline_df['x'].values
all_y = baseline_df['y'].values
print(f"\nPosition statistics:")
print(f"  X range: [{all_x.min():.2f}, {all_x.max():.2f}]")
print(f"  Y range: [{all_y.min():.2f}, {all_y.max():.2f}]")

# Check for patterns in specific N values
print("\nAnalyzing N=100 configuration:")
n100 = baseline_df[baseline_df['n'] == 100]
print(f"  Trees: {len(n100)}")
print(f"  Angle range: [{n100['deg'].min():.2f}, {n100['deg'].max():.2f}]")
print(f"  X range: [{n100['x'].min():.2f}, {n100['x'].max():.2f}]")
print(f"  Y range: [{n100['y'].min():.2f}, {n100['y'].max():.2f}]")

Analyzing baseline configuration patterns...

Angle statistics:
  Min: -28346.68
  Max: 40385.35
  Mean: 197.01
  Std: 704.17
  Unique angles (rounded to 0.1): 1925

Position statistics:
  X range: [-48.20, 3.90]
  Y range: [-4.12, 58.77]

Analyzing N=100 configuration:
  Trees: 100
  Angle range: [65.74, 249.70]
  X range: [-2.73, 2.70]
  Y range: [-2.88, 2.34]
